In [46]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchvision import models,transforms,datasets
from collections import OrderedDict
from torch.utils.data import SubsetRandomSampler
from collections import OrderedDict
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import os

In [47]:
train_data_location=os.path.join(os.getcwd(),'datasets\\data\\train\\')
test_data_location=os.path.join(os.getcwd(),'datasets\\data\\test\\')
print(test_data_location)
print(train_data_location)

E:\Developer\Machine Learning and Deep Learning\Pytorch_Training\Projects\Building\datasets\data\test\
E:\Developer\Machine Learning and Deep Learning\Pytorch_Training\Projects\Building\datasets\data\train\


In [48]:
classes=['Building','Forest','Glacier','Mountain','Sea','Street']
len(classes)

6

In [49]:

data_location='dataset/data'
batch_size=32
num_workers=1
valid_size=0.2

train_transform=transforms.Compose([transforms.RandomRotation(30),
                                        transforms.RandomResizedCrop(150),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                        ])
test_transform=transforms.Compose([transforms.Resize(150),
                                    transforms.CenterCrop(150),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                    ])

train_data=datasets.ImageFolder(data_location+'/train',transform=train_transform)
test_data=datasets.ImageFolder(data_location+'/test',transform=test_transform)

num_train=len(train_data)
indices=list(range(num_train))
np.random.shuffle(indices)
split=int(np.floor(valid_size*num_train))
train_idx,valid_idx=indices[split:],indices[:split]

train_sampler=SubsetRandomSampler(train_idx)
valid_sampler=SubsetRandomSampler(valid_idx)

trainloader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,sampler=train_sampler,num_workers=num_workers)
validloader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,sampler=valid_sampler,num_workers=num_workers)
testloader=torch.utils.data.DataLoader(test_data,batch_size=batch_size,num_workers=num_workers)

classes=['Building','Forest','Glacier','Mountain','Sea','Street']


In [54]:
trainloader.dataset

Dataset ImageFolder
    Number of datapoints: 14034
    Root location: dataset/data/train
    StandardTransform
Transform: Compose(
               RandomRotation(degrees=(-30, 30), resample=False, expand=False)
               RandomResizedCrop(size=(150, 150), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [57]:
classes=['Building','Forest','Glacier','Mountain','Sea','Street']
len_classes=len(classes)
out_fetures=len_classes
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#importing AlexNet pretrained
model=models.alexnet(pretrained=True)
#AlexNet was orignally trained for 1000 class labels, we only have 6
#replace final layer with a new one
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features,out_fetures)
#Loss Funtion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

try:
    checkpoint=torch.load('model.pt')
    print('Model Found!, Loading Model')
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    model.load_state_dict(checkpoint['model_state_dict'])
    epoch=checkpoint['epoch']
    train_loss=checkpoint['train_loss']
    valid_loss=checkpoint['valid_loss']
    print('Done!')

except:
    print('Model Not Found, Starting to train from the begining.')
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model

Model Found!, Loading Model
Done!


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [58]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for evaluation

for data, target in testloader:
    data,target=data.to(device),target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(testloader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(5):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.286845

Test Accuracy of     0: 90% (397/437)
Test Accuracy of     1: 99% (472/474)
Test Accuracy of     2: 80% (444/553)
Test Accuracy of     3: 87% (460/525)
Test Accuracy of     4: 94% (481/510)

Test Accuracy (Overall): 90% (2707/3000)


In [36]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_loader=transforms.Compose([transforms.Scale(imsize),
                          transforms.ToTensor()])
test_location='dataset/Predset/seg_pred/seg_pred/'


try:
    checkpoint=torch.load('model.pt')
    print('Model Found!, Loading Model')
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    model.load_state_dict(checkpoint['model_state_dict'])
    epoch=checkpoint['epoch']
    train_loss=checkpoint['train_loss']
    valid_loss=checkpoint['valid_loss']
    print('Done!')
except:
    print('Model Not Found')



NameError: name 'imsize' is not defined